In [2]:
import numpy as np
from matplotlib import pyplot as plt

For the cosserat rod equations, there are two kinematic equations (one related to translations and another related to rotations) and two dynamic equations (again, one related to translations and another related to rotations). So we tackle this problem in increasing levels of complexity via these milestones:

1) first tackle translations (which are easier to implement) by testing the equations on an elastic beam that is fixed on one end and has a small axial force on the one end. Because the beam is elastic, and the axial force is small, the entire beam behaves like a spring. We can then plot look at how much the beam stretches, which should correspond with analytical spring equations. This is milestone 1.

In [13]:
# This class is not completely general: only supports uniform mass and single radius, assumes element ref_lengths of 1
class cosserat_rod:
    # initialize the rod
    def __init__(self, n_elements: int = 20, positions = 0, rod_mass = 1, 
                 rod_radius = 1, G = 20, E = 40, velocities = 0, internal_forces = 0):
        
        '''Set up quantities that describe the rod'''
        self.G = G
        self.E = E
        self.n_elements = n_elements
        self.n_nodes = self.n_elements+1
        self.positions = positions if positions else np.vstack((np.arange(self.n_nodes),np.zeros((2,self.n_nodes))))
        
        self.lengths_bold = self.positions[:, 1:] - self.positions[:, :-1]
        self.lengths_norm = np.linalg.norm(self.lengths_bold, axis=0, keepdims=True)
        self.reference_lengths_bold = np.ones((3, n_elements))
        self.reference_lengths_norm = np.linalg.norm(self.reference_lengths_bold, axis=0, keepdims=True)
        
        self.directors = np.zeros((n_elements, 3, 3)) # this shape is less efficient but easier for us to think about
        for idx in range(n_elements):
            self.directors[idx, :, :] = np.eye(3)
        self.directors[:,:,0] = (self.lengths_bold/self.lengths_norm).T
        
        self.masses = rod_mass*np.ones((1,self.n_nodes))
        self.masses[[0,-1]] *= 0.5
        
        self.radius = rod_radius * np.ones((1,self.n_elements))
        self.areas = np.pi*self.radius**2     
        
        '''Set up quantities that will capture motion'''
        self.velocities = velocities if velocities else np.zeros((3, self.n_nodes)) 
        self.internal_forces = internal_forces if internal_forces else np.zeros((3, self.n_nodes))
        
        self.dilatations = self.lengths_norm / self.reference_lengths_norm
        
        self.shear_stiffness_matrix = np.zeros((n_elements, 3, 3)) # S
        alpha_c = 4.0 / 3.0
        self.shear_stiffness_matrix[:, 0, 0] = alpha_c * self.G * self.areas # S1
        self.shear_stiffness_matrix[:, 1, 1] = alpha_c * self.G * self.areas # S2
        self.shear_stiffness_matrix[:, 2, 2] = self.E * self.areas # S2
        
        self.tangents = self.lengths_bold / self.lengths_norm
        self.shear_stretch_strains = self.dilatations * self.tangents - self.directors[:, 2, :].T
    
    def rotate_rodrigues(t_frame, t_angle, about=[0.0,0.0,1.0], rad=False):
        """Rotates about one of the axes

        Parameters
        ----------
        t_frame : frame/np.array
            If frame object, then t_frame is given by the process function of
            the frame
            Else just a numpy array (vector/collection of vectors) which you
            want to rotate
        t_angle : float
            Angle of rotation, in degrees. Use `rad` to change behavior
        about : list/np.array
            Rotation axis specified in the world coordinates
        rad : bool
            Defaults to False. True indicates that the t_angle is in degrees rather
            than in radians. False indicates radians.

        Returns
        -------
        rot_frame : np.array 
            The rotated frame
        about : np.array
            The vector about which rotate_rodrigues effects rotation. Same as the
            input argument
        """
        # Check if its in radian or degree
        # Default assumed to be degree
        if not rad:
            t_angle = np.deg2rad(t_angle)

        def normalize(v):
            """ Normalize a vector/ matrix """
            norm = np.linalg.norm(v)
            if np.isclose(norm, 0.0):
                raise RuntimeError("Not rotating because axis specified to be zero")
                return v
            return v / norm

        def skew_symmetrize(v):
            """ Generate an orthogonal matrix from vector elements"""
            # Hard coded. Others are more verbose or not worth it
            return np.array([[0.0,-v[2],v[1]],
                             [v[2],0.0,-v[0]],
                             [-v[1],v[0],0.0]])


        # Convert about to np.array and normalize it
        about = normalize(np.array(about))

        # Form the 2D Euler rotation matrix
        c_angle = np.cos(t_angle)
        s_angle = np.sin(t_angle)

        # DS for 3D Euler rotation matrix
        # Composed of 2D matrices
        I = np.eye(3)
        K_mat = skew_symmetrize(about)
        # rot_matrix = I + K_mat @ (s_angle * I + (1-c_angle)* K_mat)
        rot_matrix = I + (s_angle * K_mat + (1.0 - c_angle) * (K_mat @ K_mat))   
        # print(rot_matrix, U_mat)

        if not (np.allclose(rot_matrix, I)):
            # actually do the rotation
            return rot_matrix @ t_frame, about
        else:
            return rot_matrix @ t_frame, about
            # raise RuntimeError("Not rotating because rotation is identity")
            
    # Modified trapezoidal integration
    def modified_diff(self, t_x):
            """ Modified trapezoidal integration"""
            # Pads a 0 at the end of an array
            temp = np.pad(t_x, (0,1), 'constant', constant_values=(0,0)) # Using roll calculate the diff (ghost node of 0)
            return (temp - np.roll(temp, 1))

    def position_verlet(self, dt, x, v, a):
        """Does one iteration/timestep using the Position verlet scheme

        Parameters
        ----------
        dt : float
            Simulation timestep in seconds
        x : float/array-like
            Quantity of interest / position of COM
        v : float/array-like
            Quantity of interest / velocity of COM
        force_rule : ufunc
            A function, f, that takes one argument and
            returns the instantaneous forcing

        Returns
        -------
        x_n : float/array-like
            The quantity of interest at the Next time step
        v_n : float/array-like
            The quantity of interest at the Next time step
        """
        temp_x = x + 0.5*dt*v
        v_n = v + dt * force_rule(temp_x)
        x_n = temp_x + 0.5 * dt * v_n
        return x_n, v_n

    # run the simulation, specifying external conditions ** for now it's just for first benchmark
    def run(self, ext_forces = 0, t_total = 10.0, dt = 0.1):
        #self.ext_forces = ext_forces if ext_forces else np.zeros((3, self.n_nodes))
        n_iterations = np.floor(t_total/dt)
        for time_step in range(int(n_iterations)):
            positionsHalf = self.positions + 0.5 * dt * self.velocities
            dvdt = (self.modified_diff(self.shear_stiffness_matrix @ self.shear_stretch_strains / self.dilatations) + ext_forces) / self.masses 
            self.velocities += dt*dvdt
            self.positions = positionsHalf + 0.5 * dt * self.velocities
            return self.positions


stretch_case = cosserat_rod()
stretch_case.run(np.pad(np.zeros((1,19)), (0,1), 'constant', constant_values=(0,1)))

ValueError: operands could not be broadcast together with shapes (21,4,21) (2,20) 